In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

from scipy.stats import zscore, ttest_ind

import sys
sys.path.append(r'C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis_CTR')

# Custom packages
import dataIO
import AbaTool
import GraphicTool as gt

# You can provide a "structures.json" file. If you dont, it will download one
structuresFile = r"C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis_CTR\structures.json"
A = AbaTool.Atlas(nodes=structuresFile)
DFM = AbaTool.AnatomyDataFrameManager(A)

# Load connectome data

In this cell we load an XLSX file containing the mouse connectome from Oh et al., 2014
(A mesoscale connectome of the mouse brain).  

The data used here is in **Supplementary Table 3**
> Quantitative projection strength values underlying the linear model based 
> Connectivity Matrix in figure 4a.  
> The 213 anatomical regions as both source regions (in rows) and target regions on 
> both ipsilateral and contralateral hemispheres (in columns) are shown here.

Connection strength between each area pair is given separately for the ipsilateral
and contralateral connections.  

We preprocess the data and **sum** the connection strength to both emispheres.

In [ ]:
connectomeFileName = r"D:\proj_PNN-Atlas\ABA_connectome.xlsx"
connectome = dataIO.loadConnectomeFromFile(connectomeFileName, A)

# Select only connections with the cortex (areaID: 315) as a target
connectome = connectome[315].sort_index(level=[0,1])

# connectome.head()

# Load and prepare data

In [ ]:
# --------------------------------------------------------------------
searchPath = r'D:\proj_PNN-Atlas\RESULTS\allData'
channelName = 'wfa'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# Rename 2 secondary visual areas as the old nomenclature.
# This is necessary since the ABA mouse connectome refers to an older version of 
# the Allen Atlas where some cortical areas (previously with ID:22) were not yet 
# divided in distinct subregions.
df.rename(index={417:22, 312782546:22}, inplace=True)

# Dataframe of cortical layers
cortex_df = DFM.regionsDf_to_corticalLayers(df, normalize=True, verbose=True)

# Average values across mice
layers_df = cortex_df.groupby(level='params',axis=1).mean()

layers_df

# Correlation analysis

## PNN Energy

In [ ]:
# Thalamus: 549
# Thalamus, sensory-motor cortex related: 864
# Thalamus, polymodal association cortex related: 856
sourceRegionId = 864
selectedMetric = 'energy'       # energy or diffuseFluo
# ------------------------------------------------------------------------------

# Select afferents coming from the desired source region specified by sourceRegionId
idList = connectome.index.get_level_values('mid')
filter = [A.structure_descends_from(x, sourceRegionId) for x in idList]
afferents = connectome.loc[filter].sum(axis=0)
afferents.name = 'afferents'

# Merge afferents with the cortical dataframe
metric_df = layers_df[selectedMetric].unstack('layer')
merged = metric_df.join(afferents)
# merged = merged.apply(zscore, nan_policy='omit')
merged.head()

# Select only sensory cortical areas
sensoryIds = dataIO.customSensoryAreaIds(oldAtlasNumbers=True)
merged = merged.loc[sensoryIds]
# Z-score each column
merged = merged.apply(zscore, nan_policy='omit', axis=0)
# Remove rows with a NaN
merged = merged.dropna(axis=0, how='any')
merged

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=5, figsize=(18,3), squeeze=True)
layers=['1','2/3','4','5','6']

for i, ax in enumerate(f.axes):
    gt.connectomeCorrelationScatterplot(
        merged,
        A,
        ax=ax,
        layer=layers[i],
        txtLoc='tl',
        title= 'Layer '+ layers[i],
        xlabel='Thalamic Input (z-score)' if i==0 else None,
        ylabel='PNN Energy\n(z-score)' if i==0 else None,
        fontScaling=1
        )

# plt.savefig("thalamusConnection_energy.svg", bbox_inches="tight")

## WFA Diffuse Fluorescence

In [ ]:
# Thalamus: 549
# Thalamus, sensory-motor cortex related: 864
# Thalamus, polymodal association cortex related: 856
sourceRegionId = 864
selectedMetric = 'diffuseFluo'       # energy or diffuseFluo
# ------------------------------------------------------------------------------

# Select afferents coming from the desired source region specified by sourceRegionId
idList = connectome.index.get_level_values('mid')
filter = [A.structure_descends_from(x, sourceRegionId) for x in idList]
afferents = connectome.loc[filter].sum(axis=0)
afferents.name = 'afferents'

# Merge afferents with the cortical dataframe
metric_df = layers_df[selectedMetric].unstack('layer')
merged = metric_df.join(afferents)
# merged = merged.apply(zscore, nan_policy='omit')
merged.head()

# Select only sensory cortical areas
sensoryIds = dataIO.customSensoryAreaIds(oldAtlasNumbers=True)
merged = merged.loc[sensoryIds]
# Z-score each column
merged = merged.apply(zscore, nan_policy='omit', axis=0)
# Remove rows with a NaN
merged = merged.dropna(axis=0, how='any')
merged

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=5, figsize=(18,3), squeeze=True)
layers=['1','2/3','4','5','6']

for i, ax in enumerate(f.axes):
    gt.connectomeCorrelationScatterplot(
        merged,
        A,
        ax=ax,
        layer=layers[i],
        txtLoc='tl',
        title= 'Layer '+ layers[i],
        xlabel='Thalamic Input (z-score)' if i==0 else None,
        ylabel='WFA Diffuse Fluorescence\n(z-score)' if i==0 else None,
        fontScaling=1
        )

# plt.savefig("thalamusConnection_diff.svg", bbox_inches="tight")

# Thalamic inputs of cortical subnetworks

In [ ]:
ctxGroup_df = dataIO.corticalAreasGroups(toDataFrame=True)
ctxGroup_df.index = A.acronyms_to_ids(ctxGroup_df.index)

group = ['hi' if x in ['motorSS','audioVisual','mAssociation'] else 'low' if x in ['lateral','mPrefrontal'] else 'other' for x in ctxGroup_df['function']]
ctxGroup_df['group'] = group

ctxGroup_df.head()


### Select thalamic input connectome and merge

In [ ]:
# Thalamus: 549
# Thalamus, sensory-motor cortex related: 864
# Thalamus, polymodal association cortex related: 856
sourceRegionId = 864
# ------------------------------------------------------------------------------

# Select afferents coming from the desired source region specified by sourceRegionId
idList = connectome.index.get_level_values('mid')
filter = [A.structure_descends_from(x, sourceRegionId) for x in idList]
afferents = connectome.loc[filter].sum(axis=0)
afferents.name = 'afferent'

# Merge afferents with the cortical groups dataframe
merged = ctxGroup_df.join(afferents)

### Plot the values

In [ ]:
toPlot = merged.loc[merged['group'].isin(['hi','low'])]

# Define colors
cmap = cm.get_cmap("PuBu")
barColor = cmap(0.4)

f, ax = plt.subplots(figsize=(2,7))
g = sns.barplot(
    data=toPlot,
    x = 'group',
    y = 'afferent',
    color=barColor,
    saturation=0.7,
    errorbar='se')

g.set_ylabel("Thalamic Input (A.U.)", fontsize=25)
g.set_xlabel("", fontsize=20)
ax.tick_params(labelsize=20)
ax.set_xticks([0,1],['High\nWFA','Low\nWFA'],fontsize=22)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), rotation=0, ha='center')
sns.despine()

t,p = ttest_ind(
    toPlot.loc[toPlot['group']=='hi','afferent'],
    toPlot.loc[toPlot['group']=='low','afferent'],
    nan_policy='omit'
    )
print(f"T-test - T: {t:.4f} - p-value:{p:.4}")

plt.savefig('thalamicHigh-Low.svg',bbox_inches="tight")